In [1]:
import pandas as pd
import re

In [2]:
def read_csv_files(dates, result=None):
    if result is None:
        dfs = []
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_Jakarta\\booking_{date}.csv"
            df = pd.read_csv(filename)
            dfs.append(df)
        result = pd.concat(dfs, axis=0, ignore_index=True)
    else:
        for date in dates:
            filename = f"..\\web_scrapping\\Bxl_Jakarta\\booking_{date}.csv"
            df = pd.read_csv(filename)
            result = pd.concat([result, df], axis=0, ignore_index=True)
    return result

In [3]:
dates = ["14_may_06_PM", "15_may_07_AM", "15_may_10_AM", "15_may_01_PM", "15_may_04_PM"]
df1 = read_csv_files(dates)

In [4]:
df1 = read_csv_files(["15_may_07_PM"], df1)
df1

,airline_company,dep_city,arr_city,out_dep_date,out_dep_time,out_duration,out_stop_num,out_arr_date,out_arr_time,in_dep_date,in_dep_time,in_duration,in_stop_num,in_arr_date,in_arr_time,price_ticket,hour_scrap,day_scrap
0,Qatar Airways,BRU,CGK,Aug 1,9:05 AM,17h 35m,1 stop,Aug 2,7:40 AM,Aug 15,9:05 AM,34h 05m,1 stop,Aug 16,2:10 PM,"1 521,51 €",18,14
1,Qatar Airways,BRU,CGK,Aug 1,9:05 AM,31h 25m,1 stop,Aug 2,9:30 PM,Aug 15,9:05 AM,34h 05m,1 stop,Aug 16,2:10 PM,"1 524,79 €",18,14
2,Qatar Airways,BRU,CGK,Aug 1,9:05 AM,17h 35m,1 stop,Aug 2,7:40 AM,Aug 15,7:00 PM,18h 10m,2 stops,Aug 16,8:10 AM,"1 521,51 €",18,14
3,Lufthansa,BRU,CGK,Aug 1,8:05 PM,17h 00m,2 stops,Aug 2,6:05 PM,Aug 15,7:00 PM,18h 10m,2 stops,Aug 16,8:10 AM,"2 472,62 €",18,14
4,Lufthansa,BRU,CGK,Aug 1,8:05 PM,17h 00m,2 stops,Aug 2,6:05 PM,Aug 15,7:00 PM,18h 35m,2 stops,Aug 16,8:35 AM,"2 489,71 €",18,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,Lufthansa,BRU,CGK,Aug 1,2:40 PM,22h 25m,2 stops,Aug 2,6:05 PM,Aug 15,7:00 PM,23h 00m,2 stops,Aug 16,1:00 PM,"2 691,82 €",19,15
617,"Lufthansa, operated by Cityjet For Brussels Ai...",BRU,CGK,Aug 1,7:15 PM,22h 25m,2 stops,Aug 2,10:40 PM,Aug 15,7:00 PM,23h 00m,2 stops,Aug 16,1:00 PM,"2 691,82 €",19,15
618,"Lufthansa, operated by Brussels Airlines",BRU,CGK,Aug 1,7:00 PM,22h 40m,2 stops,Aug 2,10:40 PM,Aug 15,7:00 PM,23h 00m,2 stops,Aug 16,1:00 PM,"2 685,59 €",19,15
619,Lufthansa,BRU,CGK,Aug 1,4:30 PM,25h 10m,2 stops,Aug 2,10:40 PM,Aug 15,7:00 PM,25h 50m,2 stops,Aug 16,3:50 PM,"2 485,59 €",19,15


In [5]:
df1['airline_company'] = df1['airline_company'].str.split(",").str[0]
df1['out_stop_num'] = df1['out_stop_num'].str.split(' ').str[0]
df1['in_stop_num'] = df1['in_stop_num'].str.split(' ').str[0]

split_df1 = df1['price_ticket'].str.rsplit(' ', n=1, expand=True).rename(columns={0: 'ticket_price', 1: 'currency'})
df1 = df1.iloc[:, :-2].join(split_df1).join(df1.iloc[:, -2:])
df1 = df1.drop('price_ticket', axis=1)

df1['ticket_price'] = df1['ticket_price'].str.replace(',', '.').str.replace(' ', '').astype(float)

date_cols = [col for col in df1.columns if col.endswith('_date')]
year= '2023'
for col in date_cols:
    df1[col] = pd.to_datetime(df1[col] + ' ' + year, format='%b %d %Y')

time_cols = [col for col in df1.columns if col.endswith('_time')]
df1[time_cols] = df1[time_cols].apply(lambda x: pd.to_datetime(x, format='%I:%M %p').dt.strftime('%H:%M'))

df1

,airline_company,dep_city,arr_city,out_dep_date,out_dep_time,out_duration,out_stop_num,out_arr_date,out_arr_time,in_dep_date,in_dep_time,in_duration,in_stop_num,in_arr_date,in_arr_time,ticket_price,currency,hour_scrap,day_scrap
0,Qatar Airways,BRU,CGK,2023-08-01,09:05,17h 35m,1,2023-08-02,07:40,2023-08-15,09:05,34h 05m,1,2023-08-16,14:10,1521.51,€,18,14
1,Qatar Airways,BRU,CGK,2023-08-01,09:05,31h 25m,1,2023-08-02,21:30,2023-08-15,09:05,34h 05m,1,2023-08-16,14:10,1524.79,€,18,14
2,Qatar Airways,BRU,CGK,2023-08-01,09:05,17h 35m,1,2023-08-02,07:40,2023-08-15,19:00,18h 10m,2,2023-08-16,08:10,1521.51,€,18,14
3,Lufthansa,BRU,CGK,2023-08-01,20:05,17h 00m,2,2023-08-02,18:05,2023-08-15,19:00,18h 10m,2,2023-08-16,08:10,2472.62,€,18,14
4,Lufthansa,BRU,CGK,2023-08-01,20:05,17h 00m,2,2023-08-02,18:05,2023-08-15,19:00,18h 35m,2,2023-08-16,08:35,2489.71,€,18,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,Lufthansa,BRU,CGK,2023-08-01,14:40,22h 25m,2,2023-08-02,18:05,2023-08-15,19:00,23h 00m,2,2023-08-16,13:00,2691.82,€,19,15
617,Lufthansa,BRU,CGK,2023-08-01,19:15,22h 25m,2,2023-08-02,22:40,2023-08-15,19:00,23h 00m,2,2023-08-16,13:00,2691.82,€,19,15
618,Lufthansa,BRU,CGK,2023-08-01,19:00,22h 40m,2,2023-08-02,22:40,2023-08-15,19:00,23h 00m,2,2023-08-16,13:00,2685.59,€,19,15
619,Lufthansa,BRU,CGK,2023-08-01,16:30,25h 10m,2,2023-08-02,22:40,2023-08-15,19:00,25h 50m,2,2023-08-16,15:50,2485.59,€,19,15
